In [1]:
from preprocessing import *
from postings_ner import *
import spacy
import pprint as pp

nlp = spacy.load('en_core_web_sm')

In [2]:
import pickle

pkl_file = open('postings-6-29.pkl', 'rb')
results = pickle.load(pkl_file)
pkl_file.close()

In [3]:
spacy_entity_types = {'ORG': 'Organization', 'PRODUCT': 'Product', 'CARDINAL': 'Cardinal', 'PERSON': 'Person',
                     'TIME': 'Time', 'DATE': 'Date', 'PERCENT': 'Percent', 'GPE': 'Location_Geo-Political_Area',
                     'WORK_OF_ART': 'Job_Title_or_Degree', 'LANGUAGE': 'Language', 'QUANTITY': 'Quantity',
                     'EVENT': 'Event', 'MONEY': 'Money', 'NORP': 'Nationality_or_Religion_or_Political_group',
                     'ORDINAL': 'Ordinal', 'FAC': 'Facility', 'LOC': 'Location', 'LAW': 'Law'}

stfd_entity_types = {'ORGANIZATION': 'Organization', 'TITLE': 'Title', 'LOCATION': 'Location', 'EMAIL': 'Email',
               'URL': 'URL', 'CITY': 'Location_City', 'STATE_OR_PROVINCE': 'Location_State_or_Province',
               'COUNTRY': 'Location_Country', 'NATIONALITY': 'Nationality', 'RELIGION': 'Religion', 'TITLE': 'Title',
               'IDEOLOGY': 'Ideology', 'CRIMINAL_CHARGE': 'Criminal_Charge', 'CAUSE_OF_DEATH': 'Cause_of_Death',
               'PERSON': 'Person', 'MONEY': 'Money', 'TIME': 'Time', 'ORDINAL': 'Ordinal', 'SET': 'Set',
               'DURATION': 'Duration', 'NUMBER': 'Number', 'PERCENT': 'Percent', 'MISC': 'Miscellaneous',
                'DATE': 'Date'}

In [53]:
test = 'Hello my name is Nazih Bissat and I am a data scientist intern at Snag.'

# doc = nlp(test)
# print(doc)

In [6]:
# JUST TO SEE A POSTING
def insertPeriod(position, mystring):
    longi = len(mystring)
    mystring   =  mystring[:position] + '.' + mystring[position:] 
    return mystring 

r = results[0]
posting_text = r['JD_SCRUBBED'].strip()
posting_text = re.sub(r'(\n-)', '\n', posting_text)
posting_text = re.sub(r'(\n  -)', '\n', posting_text)
posting_text = re.sub(r'(\n  -)', '\n', posting_text)
posting_text = re.sub(r'(\n  )', '\n', posting_text)

indices = [x.start() for x in re.finditer(r'\n', posting_text)]

counter = 0
for i in indices:
    if posting_text[i+counter-1] != '.':
        if posting_text[i+counter-1] != '\n':
            posting_text = insertPeriod(i+counter, posting_text)
            counter += 1
print(posting_text)

Job Description.

HUB Order Puller.

At Advance Auto Parts, a HUB Order Puller is primarily responsible for pulling HUB customer orders.

ESSENTIAL DUTIES AND RESPONSIBILITIES include the following. Other duties may be assigned.

 Pull customer orders and prepare for delivery.
 Assist with the receiving and unloading of replenishment stock.
 Store replenishment stock in proper location.
 Accurately and legibly complete all paperwork related to the task of an order puller.
 Properly maintain equipment and report any problems on a timely basis; keep work area clean and neat.
 Operate at a professional level of efficiency in order not to breech company security.
 Work with store management team to complete MAXI changes in parts department.
 Comply with all federal, state and local laws.
 Comply with all company policies and procedures.
 Complete all required training materials, attend all scheduled company meetings.

Job Requirements.

HUB Order Puller.

QUALIFICATIONS.

To perform this j

In [ ]:
# StanfordNLP NER tagger training

In [59]:
# StanfordNLP dependency parsing
startup_corenlp_server()

In [61]:
depparse = parse_posting(test)

In [63]:
pp.pprint(depparse)

{'sentences': [{'basicDependencies': [{'dep': 'ROOT',
                                       'dependent': 6,
                                       'dependentGloss': 'Bissat',
                                       'governor': 0,
                                       'governorGloss': 'ROOT'},
                                      {'dep': 'advmod',
                                       'dependent': 1,
                                       'dependentGloss': 'Hello',
                                       'governor': 6,
                                       'governorGloss': 'Bissat'},
                                      {'dep': 'nmod:poss',
                                       'dependent': 2,
                                       'dependentGloss': 'my',
                                       'governor': 3,
                                       'governorGloss': 'name'},
                                      {'dep': 'nsubj',
                                       'dependent': 3,
 

In [172]:
# CREATING POSTING TEXT FILES AND RESPECTIVE ENTITY ANNOTATION FILES FOR BRAT TEXT ANNOTATION TOOL (StanfordNLP)

def insertPeriod(position, mystring):
    longi = len(mystring)
    mystring   =  mystring[:position] + '.' + mystring[position:] 
    return mystring 

posting_index = 1
for r in results:
    posting_fname = 'posting' + str(posting_index)
    posting_text = r['JD_SCRUBBED'].strip()
    posting_text = re.sub(r'(\n-)', '\n', posting_text)
    posting_text = re.sub(r'(\n  -)', '\n', posting_text)
    posting_text = re.sub(r'(\n  -)', '\n', posting_text)
    posting_text = re.sub(r'(\n  )', '\n', posting_text)
    
    indices = [x.start() for x in re.finditer(r'\n', posting_text)]


    counter = 0
    for i in indices:
        if posting_text[i+counter-1] != '.':
            if posting_text[i+counter-1] != '\n':
                posting_text = insertPeriod(i+counter, posting_text)
                counter += 1
    
    with open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Parsing_Test/Test/' + posting_fname + '.txt', 'w') as text_file:
        text_file.write(posting_text)
        text_file.close()
    
    string_index = 1
    dep_index = 1
    parse_ann_file_text = ''
    posting_details = parse_posting(posting_text)
    word_count = 0
    for s in posting_details['sentences']:
        for t in s['tokens']:
            parse_ann_file_text += 'T' + str(string_index) + '\t' + t['pos'] + ' ' \
                                    + str(t['characterOffsetBegin']) + ' ' + str(t['characterOffsetEnd']) + '\t' \
                                    + t['word'] + '\n'
            string_index += 1
    
        for d in s['basicDependencies']:
            if d['dep'] != 'ROOT':
                parse_ann_file_text += 'R' + str(dep_index) + '\t' + d['dep'] + ' Arg1:T' \
                                        + str(d['governor'] + word_count) + ' Arg2:T' \
                                        + str(d['dependent'] + word_count) + '\n'
                dep_index += 1
    
        word_count = string_index - 1
            
    ann_file = open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Parsing_Test/Test/' + posting_fname + '.ann', 'w')
    ann_file.write(parse_ann_file_text)
    ann_file.close()
    
    posting_index += 1

In [ ]:
shutdown_corenlp_server()

In [171]:
# TEST FOR ABOVE CODE, SAME WITHOUT ITERATING OVER ALL POSTINGS, TRYING WITH INDIVIDUAL POSTING
## NEXT: TRY TO DO PARSING SENTENCE BY SENTENCE

def insertPeriod(position, mystring):
    longi = len(mystring)
    mystring   =  mystring[:position] + '.' + mystring[position:] 
    return mystring 

posting_text = results[99]['JD_SCRUBBED'].strip()
posting_text = re.sub(r'(\n-)', '\n', posting_text)
posting_text = re.sub(r'(\n  -)', '\n', posting_text)
posting_text = re.sub(r'(\n  -)', '\n', posting_text)
posting_text = re.sub(r'(\n  )', '\n', posting_text)
    
indices = [x.start() for x in re.finditer(r'\n', posting_text)]

counter = 0
for i in indices:
    if posting_text[i+counter-1] != '.':
        if posting_text[i+counter-1] != '\n':
            posting_text = insertPeriod(i+counter, posting_text)
            counter += 1
    
with open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Parsing_Test/Test/test.txt', 'w') as text_file:
        text_file.write(posting_text)
        text_file.close()
    
string_index = 1
dep_index = 1
parse_ann_file_text = ''
posting_details = parse_posting(posting_text)
word_count = 0
for s in posting_details['sentences']:
    for t in s['tokens']:
        parse_ann_file_text += 'T' + str(string_index) + '\t' + t['pos'] + ' ' \
                                    + str(t['characterOffsetBegin']) + ' ' + str(t['characterOffsetEnd']) + '\t' \
                                    + t['word'] + '\n'
        string_index += 1
    
    for d in s['basicDependencies']:
        if d['dep'] != 'ROOT':
            parse_ann_file_text += 'R' + str(dep_index) + '\t' + d['dep'] + ' Arg1:T' \
                                        + str(d['governor'] + word_count) + ' Arg2:T' \
                                        + str(d['dependent'] + word_count) + '\n'
            dep_index += 1
    
    word_count = string_index - 1
            
            
ann_file = open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Parsing_Test/Test/test.ann', 'w')
ann_file.write(parse_ann_file_text)
ann_file.close()

In [165]:
(results[99]['JD_SCRUBBED'])

'Job Description\n\nWhat is a Salesperson?\n\nEntry level sales position capable of supporting the DIY business and achieve our sales and service objectives. The role has good knowledge of store systems, basic automotive system knowledge and basic part knowledge. The role has the basic ability to source from stores, hubs, pdq, and external suppliers. The role has in-depth knowledge of the store inventory and maintenance processes. Position can be part-time or full-time. MVR certification preferred.\n\nPrimary Responsibilities\n\n  - Provide GAS2 selling experience for DIY customer visits and phone calls\n  - Achieve personal sales goal and help store achieve its sales goals\n  - Provide DIY services including battery installation, testing, wiper installs, etc.\n  - Maintain store product and operational standards\n  - Responsible for inventory processes including truck put away, shoot outs, cycle counts, Back stock, etc.\n\nSecondary Responsibilities\n\n  - Store Cleanliness including 

In [195]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(u"Autonomous cars shift insurance liability toward manufacturers")
for token in doc:
#     print(token.text, token.dep_, token.head.text, token.head.pos_,
#           [child for child in token.children])
    print(token.text, token.idx, len(token), token.dep_, token.head.text, token.head.idx, len(token.head))

Autonomous 0 10 amod cars 11 4
cars 11 4 nsubj shift 16 5
shift 16 5 ROOT shift 16 5
insurance 22 9 compound liability 32 9
liability 32 9 dobj shift 16 5
toward 42 6 prep shift 16 5
manufacturers 49 13 pobj toward 42 6


In [187]:
## CREATING TRAINING FILE FOR STANFORD NLP FROM CONLL 2003 DATA
lines = open('conll_train.txt').read().splitlines()[2:]

for i in range(len(lines)):
    if len(lines[i].split(' ')) > 1:
        lines[i] = str(lines[i].split(' ')[0]) + ' ' + str(lines[i].split(' ')[3])

open('conll_train_stfd_nlp.txt','w').write('\n'.join(lines))

1668940

In [217]:
##### COMMAND TO TRAIN STANFORDNLP NER: java -cp "stanford-ner.jar:lib/*" -mx4g edu.stanford.nlp.ie.crf.CRFClassifier -prop train/prop.txt
from subprocess import Popen, PIPE

commands = '''
cd /Users/nazih.bissat/Desktop/match.fracking/stanford-ner-tagger;
java -cp "stanford-ner.jar:lib/*" -mx4g edu.stanford.nlp.ie.crf.CRFClassifier -prop train_conll/prop.txt
'''

process = Popen('/bin/bash', stdin=PIPE, stdout=PIPE)
out, err = process.communicate(commands.encode('utf-8'))

# full_entities = out.decode('utf-8')

In [8]:
## PREPROCESSING TO FORMAT TRAINING DATA FOR SPACY
lines = open('stanford-ner-tagger/train_conll/conll_train_stfd_nlp.tsv').read().splitlines()[2:]

TRAIN_DATA = list()
char_count = 0
sent = ''
ents = list()
for i in range(len(lines)):
    if len(lines[i].split(' ')) > 1:
        word, entity_type = lines[i].split(' ')[0], lines[i].split(' ')[1]
        sent += word + ' '
        if entity_type != 'O':
            ents.append((char_count, char_count + len(word), entity_type))
        char_count += len(word) + 1
    else:
        TRAIN_DATA.append((sent, {'entities': ents}))
        sent = ''
        char_count = 0
        ents = list()
    

In [6]:
print(TRAIN_DATA[0:5])

[('German call to boycott British lamb . ', {'entities': [(0, 6, 'I-MISC'), (23, 30, 'I-MISC')]}), ('Peter Blackburn ', {'entities': [(0, 5, 'I-PER'), (6, 15, 'I-PER')]}), ('BRUSSELS 1996-08-22 ', {'entities': [(0, 8, 'I-LOC')]}), ('The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep . ', {'entities': [(4, 12, 'I-ORG'), (13, 23, 'I-ORG'), (59, 65, 'I-MISC'), (94, 101, 'I-MISC')]}), ("Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . ", {'entities': [(0, 7, 'I-LOC'), (33, 41, 'I-ORG'), (42, 47, 'I-ORG'), (72, 78, 'I-PER'), (79, 88, 'I-PER'), (164, 171, 'I-LOC')]})]


In [16]:
### EXAMPLE CODE TO TRAIN SPACY MODEL
import random
from __future__ import unicode_literals, print_function

import plac
from pathlib import Path

# TRAIN_DATA = [
#      ("Uber blew through $1 million a week", {'entities': [(0, 4, 'ORG')]}),
#      ("Google rebrands its business apps", {'entities': [(0, 6, "ORG")]})]

output_dir = 'spaCy_conll_model'

nlp = spacy.blank('en')
optimizer = nlp.begin_training()
for i in range(20):
    random.shuffle(TRAIN_DATA)
    for text, annotations in TRAIN_DATA:
        nlp.update([text], [annotations], sgd=optimizer)

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

    # test the saved model
    print("Loading from", output_dir)
    nlp2 = spacy.load(output_dir)
    for text, _ in TRAIN_DATA[0:50]:
        doc = nlp2(text)
        if doc.ents != []:
            print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
            print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Saved model to spaCy_conll_model
Loading from spaCy_conll_model
Entities []
Tokens [('Orii', '', 0), ('-', '', 0), ('95/96', '', 0), ('group', '', 0), ('results', '', 0), ('.', '', 0)]
Entities []
Tokens [('7', '', 0), ('-', '', 0), ('Todd', '', 0), ('Martin', '', 0), ('(', '', 0), ('U.S.', '', 0), (')', '', 0), ('beat', '', 0), ('9', '', 0), ('-', '', 0), ('Cedric', '', 0), ('Pioline', '', 0), ('(', '', 0), ('France', '', 0), (')', '', 0), ('2', '', 0), ('-', '', 0), ('6', '', 0), ('6', '', 0), ('-', '', 0), ('2', '', 0)]
Entities []
Tokens [('Results', '', 0), ('in', '', 0), ('the', '', 0), ('Malaysian', '', 0)]
Entities []
Tokens [('(', '', 0), ('players', '', 0), ('U.S.', '', 0), ('unless', '', 0), ('stated', '', 0), (')', '', 0), (':', '', 0)]
Entities []
Tokens [('Oval', '', 0), ('on', '', 0), ('Sunday', '', 0), (':', '', 0)]
Entities []
Tokens [('Palestinians', '', 0), ('to', '', 0), ('strike', '', 0), ('over', '', 0), ('Jerusalem', '', 0), ('demolition', '', 0), ('.', '', 0)]
E

Entities []
Tokens [('3', '', 0), ('.', '', 0), ('Lance', '', 0), ('Armstrong', '', 0), ('(', '', 0), ('U.S.', '', 0), (')', '', 0), ('Motorola', '', 0)]
Entities []
Tokens [('LONDON', '', 0), ('1996', '', 0), ('-', '', 0), ('08', '', 0), ('-', '', 0), ('30', '', 0)]
Entities []
Tokens [('He', '', 0), ('praised', '', 0), ('Clinton', '', 0), ('for', '', 0), ('vetoing', '', 0), ('the', '', 0), ('Republican', '', 0), ('Congress', '', 0), ("'", '', 0), ('attempt', '', 0), ('to', '', 0), ('repeal', '', 0), ('parts', '', 0), ('of', '', 0), ('the', '', 0), ('Clean', '', 0), ('Air', '', 0), ('Act', '', 0), (',', '', 0), ('and', '', 0), ('criticized', '', 0), ('Republicans', '', 0), ('for', '', 0), ('"', '', 0), ('slashing', '', 0), ('money', '', 0), ('to', '', 0), ('combat', '', 0), ('drugs', '', 0), ('in', '', 0), ('schools', '', 0), ('.', '', 0), ('"', '', 0)]
Entities []
Tokens [('BRUSSELS', '', 0), ('1996', '', 0), ('-', '', 0), ('08', '', 0), ('-', '', 0), ('23', '', 0)]
Entities []
Token

In [22]:
print(len(TRAIN_DATA[0:1000]))

1000


In [19]:
### OR

#!/usr/bin/env python
# coding: utf8
"""Example of training spaCy's named entity recognizer, starting off with an
existing model or a blank model.

For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities

Compatible with: spaCy v2.0.0+
"""
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy

# import multiprocessing

# multiprocessing.cpu_count()


# training data
# TRAIN_DATA = [
#     ('Who is Shaka Khan?', {
#         'entities': [(7, 17, 'PERSON')]
#     }),
#     ('I like London and Berlin.', {
#         'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
#     })
# ]


# @plac.annotations(
#     model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
#     output_dir=("Optional output directory", "option", "o", Path),
#     n_iter=("Number of training iterations", "option", "n", int))
def main(model=None, output_dir=None, n_iter=5):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
        print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
            print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])


# if __name__ == '__main__':
#     plac.call(main)

    # Expected output:
    # Entities [('Shaka Khan', 'PERSON')]
    # Tokens [('Who', '', 2), ('is', '', 2), ('Shaka', 'PERSON', 3),
    # ('Khan', 'PERSON', 1), ('?', '', 2)]
    # Entities [('London', 'LOC'), ('Berlin', 'LOC')]
    # Tokens [('I', '', 2), ('like', '', 2), ('London', 'LOC', 3),
    # ('and', '', 2), ('Berlin', 'LOC', 3), ('.', '', 2)]
    
main(output_dir='spaCy_CONLL')

Created blank 'en' model


/Users/nazih.bissat/anaconda3/envs/myenv1/lib/python3.6/site-packages/numpy/linalg/linalg.py:2257: RuntimeWarning: invalid value encountered in sqrt
  ret = sqrt(sqnorm)


{'ner': 30561.572949256533}


KeyboardInterrupt: 

In [26]:
test = '''EU I-ORG
rejects O
German I-MISC
call O
to O
boycott O
British I-MISC
lamb O
. O

Peter I-PER
Blackburn I-PER

BRUSSELS I-LOC
1996-08-22 O

The O
European I-ORG
Commission I-ORG
said O
on O
Thursday O
it O
disagreed O
with O
German I-MISC
advice O
to O
consumers O
to O
shun O
British I-MISC
lamb O
until O
scientists O
determine O
whether O
mad O
cow O
disease O
can O
be O
transmitted O
to O
sheep O
. O

Germany I-LOC
's O
representative O
to O
the O
European I-ORG
Union I-ORG
's O
veterinary O
committee O
Werner I-PER
Zwingmann I-PER
said O
on O
Wednesday O
consumers O
should O
buy O
sheepmeat O
from O
countries O
other O
than O
Britain I-LOC
until O
the O
scientific O
advice O
was O
clearer O
. O

" O
We O
do O
n't O
support O
any O
such O
recommendation O
because O
we O
do O
n't O
see O
any O
grounds O
for O
it O
, O
" O
the O
Commission I-ORG
's O
chief O
spokesman O
Nikolaus I-PER
van I-PER
der I-PER
Pas I-PER
told O
a O
news O
briefing O
. O'''

text = ''
for line in test.split('\n'):
    if line == '':
        text += '\n'
    else:
        text += line.split(' ')[0] + ' '
    
print(text)

EU rejects German call to boycott British lamb . 
Peter Blackburn 
BRUSSELS 1996-08-22 
The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep . 
Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . 
" We do n't support any such recommendation because we do n't see any grounds for it , " the Commission 's chief spokesman Nikolaus van der Pas told a news briefing . 


In [34]:
## THIS CELL HAS CODE TO TEST A TRAINED NER MODEL
from subprocess import Popen, PIPE

commands = '''
cd /Users/nazih.bissat/Desktop/match.fracking/stanford-ner-tagger;
java -cp stanford-ner.jar edu.stanford.nlp.ie.crf.CRFClassifier -loadClassifier conll-train-ner-model.ser.gz -testFile test.tsv
'''

process = Popen('/bin/bash', stdin=PIPE, stdout=PIPE)
out, err = process.communicate(commands.encode('utf-8'))

full_entities = out.decode('utf-8')
print(out.decode('utf-8'))

EU	I-ORG	I-ORG
rejects	O	O
German	I-MISC	I-MISC
call	O	O
to	O	O
boycott	O	O
British	I-MISC	I-MISC
lamb	O	O
.	O	O

Peter	I-PER	I-PER
Blackburn	I-PER	I-PER

BRUSSELS	I-LOC	I-LOC
1996-08-22	O	O

The	O	O
European	I-ORG	I-ORG
Commission	I-ORG	I-ORG
said	O	O
on	O	O
Thursday	O	O
it	O	O
disagreed	O	O
with	O	O
German	I-MISC	I-MISC
advice	O	O
to	O	O
consumers	O	O
to	O	O
shun	O	O
British	I-MISC	I-MISC
lamb	O	O
until	O	O
scientists	O	O
determine	O	O
whether	O	O
mad	O	O
cow	O	O
disease	O	O
can	O	O
be	O	O
transmitted	O	O
to	O	O
sheep	O	O
.	O	O

Germany	I-LOC	I-LOC
's	O	O
representative	O	O
to	O	O
the	O	O
European	I-ORG	I-ORG
Union	I-ORG	I-ORG
's	O	O
veterinary	O	O
committee	O	O
Werner	I-PER	I-PER
Zwingmann	I-PER	I-PER
said	O	O
on	O	O
Wednesday	O	O
consumers	O	O
should	O	O
buy	O	O
sheepmeat	O	O
from	O	O
countries	O	O
other	O	O
than	O	O
Britain	I-LOC	I-LOC
until	O	O
the	O	O
scientific	O	O
advice	O	O
was	O	O
clearer	O	O
.	O	O

"	O	O
We	O	O
do	O	O
n't	O	O
support	O	O
any	O	O
such	O	O
recommendation	O	O


In [ ]:
# Re-train StanfordNLP NER model from brat
train_data = ''

for filename in os.listdir('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/NER/StanfordNLP'):
    for i in np.arange(1, 101, 1):
        if filename.startswith("posting" + str(i)): 
            # print(os.path.join(directory, filename))
            continue
        else:
            continue

In [58]:
train_data = ''

posting = '''Job Description

HUB Order Puller

At Advance Auto Parts, a HUB Order Puller is primarily responsible for pulling HUB customer orders.

ESSENTIAL DUTIES AND RESPONSIBILITIES include the following. Other duties may be assigned.

  - Pull customer orders and prepare for delivery.
  - Assist with the receiving and unloading of replenishment stock.
  - Store replenishment stock in proper location.
  - Accurately and legibly complete all paperwork related to the task of an order puller.
  - Properly maintain equipment and report any problems on a timely basis; keep work area clean and neat.
  - Operate at a professional level of efficiency in order not to breech company security.
  - Work with store management team to complete MAXI changes in parts department.
  - Comply with all federal, state and local laws.
  - Comply with all company policies and procedures.
  - Complete all required training materials, attend all scheduled company meetings.

Job Requirements

HUB Order Puller

QUALIFICATIONS

To perform this job successfully, an individual must be able to perform each essential duty satisfactorily. The requirements listed are representative of the knowledge, skill, and/or ability required. Reasonable accommodations may be made to enable individuals with disabilities to perform the essential functions.

  - Able to use a hand truck and pallet jack.
  - Able to follow detailed instructions.
  - Able to work efficiently with others both within and outside the store.
  - Able to perform successfully with minimum supervision.
  - Maintain a good personal appearance.
  - Scheduling factors necessitate that the individual should be able to come to work on short notice, and/or work flexible hours including nights and weekends.

EDUCATION and/or EXPERIENCE

  - High school diploma or GED equivalent; or one to three months related experience and/or training; or equivalent combination or education and experience.

LANGUAGE SKILLS

  - Ability to read and comprehend simple instruction, short correspondence, and memos.
  - Ability to write simple correspondence.
  - Ability to effectively present information in one on one and small group situations to customer, clients and other Team Members of the organization.

MATHEMATICAL SKILLS

  - Ability to add, subtract, multiply, and divide in all units of measure, using whole numbers, common fractions, and decimals.
  - Ability to compute rates, ratios, percentages and interprets bar graphs.

REASONING ABILITY

  - Ability to define problems collects and evaluates information, establish facts and draw valid conclusions.
  - Ability to interpret Advance policies and'''

anns = '''T1	Organization 732 736	MAXI
T2	Title 1144 1158	representative
T3	Duration 1825 1828	one
T4	Duration 1832 1844	three months
T5	Number 2136 2139	one
T6	Number 2143 2146	one'''

def insertPeriod(position, mystring):
    longi = len(mystring)
    mystring   =  mystring[:position] + '.' + mystring[position:] 
    return mystring 

posting = posting.strip()
posting = re.sub(r'(\n-)', '\n', posting)
posting = re.sub(r'(\n  -)', '\n', posting)
posting = re.sub(r'(\n  -)', '\n', posting)
posting = re.sub(r'(\n  )', '\n', posting)
    
indices = [x.start() for x in re.finditer(r'\n', posting)]

counter = 0
for i in indices:
    if posting[i+counter-1] != '.':
        if posting[i+counter-1] != '\n':
            posting = insertPeriod(i+counter, posting)
            counter += 1
            
tokens = posting.split(' ')

ents = [ann.split('\t')[1].split(' ') for ann in anns.split('\n')]

counter = 0
for t in tokens:
    ent = False
    for i in ents:
        if (counter == int(i[1])):
            train_data += t + ' ' + i[0] + '\n'
            ent = True
    if ent == False:
        train_data += t + ' ' + 'O' + '\n'
    counter += len(t) + 1

In [60]:
print(tokens)

['Job', 'Description.\n\nHUB', 'Order', 'Puller.\n\nAt', 'Advance', 'Auto', 'Parts,', 'a', 'HUB', 'Order', 'Puller', 'is', 'primarily', 'responsible', 'for', 'pulling', 'HUB', 'customer', 'orders.\n\nESSENTIAL', 'DUTIES', 'AND', 'RESPONSIBILITIES', 'include', 'the', 'following.', 'Other', 'duties', 'may', 'be', 'assigned.\n\n', 'Pull', 'customer', 'orders', 'and', 'prepare', 'for', 'delivery.\n', 'Assist', 'with', 'the', 'receiving', 'and', 'unloading', 'of', 'replenishment', 'stock.\n', 'Store', 'replenishment', 'stock', 'in', 'proper', 'location.\n', 'Accurately', 'and', 'legibly', 'complete', 'all', 'paperwork', 'related', 'to', 'the', 'task', 'of', 'an', 'order', 'puller.\n', 'Properly', 'maintain', 'equipment', 'and', 'report', 'any', 'problems', 'on', 'a', 'timely', 'basis;', 'keep', 'work', 'area', 'clean', 'and', 'neat.\n', 'Operate', 'at', 'a', 'professional', 'level', 'of', 'efficiency', 'in', 'order', 'not', 'to', 'breech', 'company', 'security.\n', 'Work', 'with', 'store', 

In [48]:
counter = 0
for t in tokens:
    for i in ents:
        if (counter == int(i[0])):
            print(i[0])
            print(t)
    counter += len(t) + 1

732
MAXI
1144
representative
1825
one
1832
three
2136
one
2143
one


In [62]:
# A python script to turn annotated data in standoff format (brat annotation tool) to the formats expected by Stanford NER and Relation Extractor models
# - NER format based on: http://nlp.stanford.edu/software/crf-faq.html#a

import os
from os import listdir
from os.path import isfile, join

DEFAULT_OTHER_ANNO = 'O'
DATA_DIRECTORY = '/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/NER/StanfordNLP'
OUTPUT_DIRECTORY = 'stanford-nlp-train-model'

NER_TRAINING_DATA_OUTPUT_PATH = join(OUTPUT_DIRECTORY, 'ner-crf-training-data.tsv')

if os.path.exists(OUTPUT_DIRECTORY):
    if os.path.exists(NER_TRAINING_DATA_OUTPUT_PATH):
        os.remove(NER_TRAINING_DATA_OUTPUT_PATH)
else:
    os.makedirs(OUTPUT_DIRECTORY)

sentence_count = 0
# nlp = StanfordCoreNLP(CORENLP_SERVER_ADDRESS)
startup_corenlp_server()

# looping through .ann files in the data directory
ann_data_files = [f for f in listdir(DATA_DIRECTORY) if isfile(join(DATA_DIRECTORY, f)) and f.split('.')[1] == 'ann']

for file in ann_data_files:
    entities = []

    # process .ann file - place entities and relations into 2 seperate lists of tuples
    with open(join(DATA_DIRECTORY, file), 'r') as document_anno_file:
        lines = document_anno_file.readlines()
        for line in lines:
            standoff_line = line.split()
            entity = {}
            entity['standoff_id'] = int(standoff_line[0][1:])
            entity['entity_type'] = standoff_line[1].capitalize()
            entity['offset_start'] = int(standoff_line[2])
            entity['offset_end'] = int(standoff_line[3])
            entity['word'] = standoff_line[4]
            entities.append(entity)

    # read the .ann's matching .txt file and tokenize its text using stanford corenlp
    with open(join(DATA_DIRECTORY, file.replace('.ann', '.txt')), 'r') as document_text_file:
        document_text = document_text_file.read()

    output = annotate_posting(document_text)

    # write text and annotations into NER
    with open(NER_TRAINING_DATA_OUTPUT_PATH, 'a') as ner_training_data:
        for sentence in output['sentences']:
            entities_in_sentence = {}
            sentence_re_rows = []

            for token in sentence['tokens']:
                offset_start = int(token['characterOffsetBegin'])
                offset_end = int(token['characterOffsetEnd'])

                re_row = {}
                entity_found = False
                ner_anno = DEFAULT_OTHER_ANNO

                # searching for token in annotated entities
                for entity in entities:
                    if offset_start >= entity['offset_start'] and offset_end <= entity['offset_end']:
                        ner_anno = entity['entity_type']

                    # multi-token entities for RE need to be handled differently than NER
                    if offset_start == entity['offset_start'] and offset_end <= entity['offset_end']:
                        entities_in_sentence[entity['standoff_id']] = len(sentence_re_rows)
                        re_row['entity_type'] = entity['entity_type']
                        re_row['pos_tag'] = token['pos']
                        re_row['word'] = token['word']

                        sentence_re_rows.append(re_row)
                        entity_found = True
                        break
                    elif offset_start > entity['offset_start'] and offset_end <= entity['offset_end'] and len(
                            sentence_re_rows) > 0:
                        sentence_re_rows[-1]['pos_tag'] += '/{}'.format(token['pos'])
                        sentence_re_rows[-1]['word'] += '/{}'.format(token['word'])
                        entity_found = True
                        break

                if not entity_found:
                    re_row['entity_type'] = DEFAULT_OTHER_ANNO
                    re_row['pos_tag'] = token['pos']
                    re_row['word'] = token['word']

                    sentence_re_rows.append(re_row)

                # writing tagged tokens to NER training data
                ner_training_data.write('{}\t{}\n'.format(token['word'], ner_anno))

            sentence_count += 1

        ner_training_data.write('\n')

    print('Processed file pair: {} and {}'.format(file, file.replace('.ann', '.txt')))

Processed file pair: posting16.ann and posting16.txt
Processed file pair: posting17.ann and posting17.txt
Processed file pair: posting29.ann and posting29.txt
Processed file pair: posting15.ann and posting15.txt
Processed file pair: posting100.ann and posting100.txt
Processed file pair: posting14.ann and posting14.txt
Processed file pair: posting28.ann and posting28.txt
Processed file pair: posting10.ann and posting10.txt
Processed file pair: posting38.ann and posting38.txt
Processed file pair: posting39.ann and posting39.txt
Processed file pair: posting11.ann and posting11.txt
Processed file pair: posting13.ann and posting13.txt
Processed file pair: posting12.ann and posting12.txt
Processed file pair: posting49.ann and posting49.txt
Processed file pair: posting61.ann and posting61.txt
Processed file pair: posting75.ann and posting75.txt
Processed file pair: posting74.ann and posting74.txt
Processed file pair: posting60.ann and posting60.txt
Processed file pair: posting48.ann and posti

In [ ]:
## DON'T RUN YET, THIS TAKES A TEXT FILE, RUNS A SPECIFIED NER MODEL ON IT, AND OUTPUTS TO A SPECIFIED OUTPUT DIRECTORY
from subprocess import Popen, PIPE

commands = '''
cd /Users/nazih.bissat/Desktop/match.fracking/stanford-ner-tagger;
java -cp stanford-ner.jar edu.stanford.nlp.ie.crf.CRFClassifier -loadClassifier conll-train-ner-model.ser.gz -outputFormat inlineXML -textFile untitled.txt > test_file.tsv
'''

process = Popen('/bin/bash', stdin=PIPE, stdout=PIPE)
out, err = process.communicate(commands.encode('utf-8'))

full_entities = out.decode('utf-8')
print(out.decode('utf-8'))

In [28]:
## TEST TO CONVERT OUTPUT OF STANFORD NLP NER MODEL TO BRAT STANDOFF FORMAT (1)
test = '<I-ORG>HUB</I-ORG> Order <I-PER>Puller</I-PER>.'
sent = re.sub(r'\<(.*?)>', '', test)
print(test)
print(sent)

regex = re.compile(r'\<(.*?)>')
iterator = regex.finditer(test)

ind = list()
for i in iterator:
    ind.append(i.span())

print(ind)

<I-ORG>HUB</I-ORG> Order <I-PER>Puller</I-PER>.
HUB Order Puller.
[(0, 7), (10, 18), (25, 32), (38, 46)]


In [38]:
## TEST TO CONVERT OUTPUT OF STANFORD NLP NER MODEL TO BRAT STANDOFF FORMAT (2)
import numpy as np

test = '<I-ORG>HUB</I-ORG> Order <I-PER>Puller</I-PER>.'
sent = re.sub(r'\<(.*?)>', '', test)

regex = re.compile(r'\<(.*?)>')
iterator = regex.finditer(test)

ind = list()
for i in iterator:
    ind.append(i.span())
    
counter = 0
for i in np.arange(0, len(ind), 2):
    print('Entity:', sent[(ind[i][0] - counter):(ind[i+1][0] + ind[i][0] - ind[i][1] - counter)])
    print('startChar:', (ind[i][0] - counter))
    print('endChar:', ind[i+1][0] + ind[i][0] - ind[i][1] - counter)
    print()
    counter += 2 * (ind[i][1] - ind[i][0]) + 1

Entity: HUB
startChar: 0
endChar: 3

Entity: Puller
startChar: 10
endChar: 16

